In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True, reshape=False)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.00001
epochs = 10
batch_size = 128
test_valid_size = 256
n_classes = 10
dropout = 0.75



In [3]:
def conv2d(x, w, b):
    x = tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x):
    return tf.nn.max_pool(
        x, 
        ksize=[1,2,2,1], 
        strides=[1,2,2,1], 
        padding='SAME')

# def maxpool2d(x, k=2):
#     return tf.nn.max_pool(
#         x,
#         ksize=[1, k, k, 1],
#         strides=[1, k, k, 1],
#         padding='SAME')
    
def conv_net(x, weights, biases, dropout):
    # conv 1
    x = conv2d(x, weights['wc1'], biases['bc1'])
    x = maxpool2d(x)
    
    # conv 2
    x = conv2d(x, weights['wc2'], biases['bc2'])
    x = maxpool2d(x)
    
    # fully connected 1
    x = tf.reshape(x, [-1, weights['wd1'].get_shape().as_list()[0]])
    x = tf.matmul(x, weights['wd1'])
    x = tf.add(x, biases['bd1'])
    x = tf.nn.relu(x)
    x = tf.nn.dropout(x, dropout)
    
    # output
    x = tf.matmul(x, weights['out'])
    return tf.add(x, biases['out'])
    

weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}    
    

# weights = {
#     # conv: stride of 1, use VALID
#     # pooling: stride of 2, use VALID
#     # ceil((28 - 2 + 1)/1) = 27 --> ceil((27- 2 + 1)/2) --> 13,13,32
#     'wc1': tf.Variable(tf.random_normal([2,2,1,32])), 
#     # ceil((13 - 2 + 1)/1) = 12 --> ceil((12 - 2 + 1)/2) --> 6,6,64
#     'wc2': tf.Variable(tf.random_normal([2,2,32,64])),
#     'wd1': tf.Variable(tf.random_normal([6*6*64,1024])),
#     'out': tf.Variable(tf.random_normal([1024, n_classes]))
# }


# biases = {
#     'bc1': tf.Variable(tf.random_normal([32])),
#     'bc2': tf.Variable(tf.random_normal([64])),
#     'bd1': tf.Variable(tf.random_normal([1024])),
#     'out': tf.Variable(tf.random_normal([n_classes]))
# }




In [4]:
# tf graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)


# model
logits = conv_net(x, weights, biases, keep_prob)

# loss and optimizer
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

optimizer = tf.train.GradientDescentOptimizer(
    learning_rate=learning_rate).minimize(cost)

# accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            session.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})
            
            loss = session.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob:1.})
            
            valid_acc = session.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob:1.})
            
            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format( 
                 epoch+1,
                 batch+1,
                 loss,
                 valid_acc))
            
    test_acc = session.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob:1.})
    print('Testing Accuracy: {}'.format(test_acc))
        



Epoch  1, Batch   1 - Loss: 72288.4219 Validation Accuracy: 0.082031
Epoch  1, Batch   2 - Loss: 49780.0508 Validation Accuracy: 0.097656
Epoch  1, Batch   3 - Loss: 37919.0508 Validation Accuracy: 0.117188
Epoch  1, Batch   4 - Loss: 34196.5312 Validation Accuracy: 0.125000
Epoch  1, Batch   5 - Loss: 31522.9414 Validation Accuracy: 0.144531
Epoch  1, Batch   6 - Loss: 28819.0762 Validation Accuracy: 0.183594
Epoch  1, Batch   7 - Loss: 26124.1934 Validation Accuracy: 0.179688
Epoch  1, Batch   8 - Loss: 23882.7637 Validation Accuracy: 0.210938
Epoch  1, Batch   9 - Loss: 24180.9238 Validation Accuracy: 0.207031
Epoch  1, Batch  10 - Loss: 22268.5547 Validation Accuracy: 0.195312
Epoch  1, Batch  11 - Loss: 17439.1328 Validation Accuracy: 0.183594
Epoch  1, Batch  12 - Loss: 18781.0039 Validation Accuracy: 0.226562
Epoch  1, Batch  13 - Loss: 21319.6504 Validation Accuracy: 0.222656
Epoch  1, Batch  14 - Loss: 18591.2852 Validation Accuracy: 0.214844
Epoch  1, Batch  15 - Loss: 14377.

Epoch  1, Batch 120 - Loss:  3851.5718 Validation Accuracy: 0.601562
Epoch  1, Batch 121 - Loss:  4606.7500 Validation Accuracy: 0.609375
Epoch  1, Batch 122 - Loss:  3200.2588 Validation Accuracy: 0.605469
Epoch  1, Batch 123 - Loss:  3835.8384 Validation Accuracy: 0.609375
Epoch  1, Batch 124 - Loss:  3127.1021 Validation Accuracy: 0.605469
Epoch  1, Batch 125 - Loss:  2586.2217 Validation Accuracy: 0.613281
Epoch  1, Batch 126 - Loss:  3952.4548 Validation Accuracy: 0.597656
Epoch  1, Batch 127 - Loss:  4050.4304 Validation Accuracy: 0.597656
Epoch  1, Batch 128 - Loss:  4534.5439 Validation Accuracy: 0.597656
Epoch  1, Batch 129 - Loss:  4427.4150 Validation Accuracy: 0.582031
Epoch  1, Batch 130 - Loss:  3501.4519 Validation Accuracy: 0.589844
Epoch  1, Batch 131 - Loss:  3100.8198 Validation Accuracy: 0.605469
Epoch  1, Batch 132 - Loss:  2889.1572 Validation Accuracy: 0.605469
Epoch  1, Batch 133 - Loss:  3382.9683 Validation Accuracy: 0.601562
Epoch  1, Batch 134 - Loss:  3224.

Epoch  1, Batch 239 - Loss:  2166.5667 Validation Accuracy: 0.691406
Epoch  1, Batch 240 - Loss:  2231.5085 Validation Accuracy: 0.703125
Epoch  1, Batch 241 - Loss:  1946.7653 Validation Accuracy: 0.703125
Epoch  1, Batch 242 - Loss:  2408.2021 Validation Accuracy: 0.710938
Epoch  1, Batch 243 - Loss:  1987.7574 Validation Accuracy: 0.707031
Epoch  1, Batch 244 - Loss:  2444.5732 Validation Accuracy: 0.714844
Epoch  1, Batch 245 - Loss:  1960.9260 Validation Accuracy: 0.710938
Epoch  1, Batch 246 - Loss:  2352.0903 Validation Accuracy: 0.703125
Epoch  1, Batch 247 - Loss:  2599.9929 Validation Accuracy: 0.703125
Epoch  1, Batch 248 - Loss:  2602.7520 Validation Accuracy: 0.703125
Epoch  1, Batch 249 - Loss:  1905.5813 Validation Accuracy: 0.699219
Epoch  1, Batch 250 - Loss:  2532.0020 Validation Accuracy: 0.707031
Epoch  1, Batch 251 - Loss:  1712.0872 Validation Accuracy: 0.703125
Epoch  1, Batch 252 - Loss:  2674.7429 Validation Accuracy: 0.703125
Epoch  1, Batch 253 - Loss:  3033.

Epoch  1, Batch 358 - Loss:  1331.8829 Validation Accuracy: 0.710938
Epoch  1, Batch 359 - Loss:  1788.6680 Validation Accuracy: 0.718750
Epoch  1, Batch 360 - Loss:  1233.8401 Validation Accuracy: 0.726562
Epoch  1, Batch 361 - Loss:  1776.8174 Validation Accuracy: 0.730469
Epoch  1, Batch 362 - Loss:  1971.2635 Validation Accuracy: 0.730469
Epoch  1, Batch 363 - Loss:  1516.0756 Validation Accuracy: 0.726562
Epoch  1, Batch 364 - Loss:  1471.9937 Validation Accuracy: 0.718750
Epoch  1, Batch 365 - Loss:  1446.5870 Validation Accuracy: 0.722656
Epoch  1, Batch 366 - Loss:  1645.0669 Validation Accuracy: 0.726562
Epoch  1, Batch 367 - Loss:  1309.5349 Validation Accuracy: 0.718750
Epoch  1, Batch 368 - Loss:  2345.9126 Validation Accuracy: 0.722656
Epoch  1, Batch 369 - Loss:  1873.4866 Validation Accuracy: 0.722656
Epoch  1, Batch 370 - Loss:  2005.5719 Validation Accuracy: 0.722656
Epoch  1, Batch 371 - Loss:  1660.0446 Validation Accuracy: 0.722656
Epoch  1, Batch 372 - Loss:  1485.

Epoch  2, Batch  48 - Loss:  1918.7046 Validation Accuracy: 0.753906
Epoch  2, Batch  49 - Loss:  1346.8708 Validation Accuracy: 0.750000
Epoch  2, Batch  50 - Loss:  1389.6367 Validation Accuracy: 0.746094
Epoch  2, Batch  51 - Loss:  1707.9500 Validation Accuracy: 0.750000
Epoch  2, Batch  52 - Loss:   989.9395 Validation Accuracy: 0.757812
Epoch  2, Batch  53 - Loss:   891.0648 Validation Accuracy: 0.750000
Epoch  2, Batch  54 - Loss:  1164.2357 Validation Accuracy: 0.753906
Epoch  2, Batch  55 - Loss:  1169.2845 Validation Accuracy: 0.757812
Epoch  2, Batch  56 - Loss:  1431.5474 Validation Accuracy: 0.753906
Epoch  2, Batch  57 - Loss:  2043.9041 Validation Accuracy: 0.753906
Epoch  2, Batch  58 - Loss:  1525.2263 Validation Accuracy: 0.753906
Epoch  2, Batch  59 - Loss:  1222.1251 Validation Accuracy: 0.753906
Epoch  2, Batch  60 - Loss:  1490.1954 Validation Accuracy: 0.750000
Epoch  2, Batch  61 - Loss:  1005.5691 Validation Accuracy: 0.750000
Epoch  2, Batch  62 - Loss:  1450.

...

Epoch  5, Batch 427 - Loss:   406.2811 Validation Accuracy: 0.808594
Epoch  5, Batch 428 - Loss:   362.1173 Validation Accuracy: 0.812500
Epoch  5, Batch 429 - Loss:   570.3937 Validation Accuracy: 0.808594
Epoch  6, Batch   1 - Loss:   403.9369 Validation Accuracy: 0.808594
Epoch  6, Batch   2 - Loss:   371.4293 Validation Accuracy: 0.808594
Epoch  6, Batch   3 - Loss:   633.4845 Validation Accuracy: 0.804688
Epoch  6, Batch   4 - Loss:   429.1577 Validation Accuracy: 0.804688
Epoch  6, Batch   5 - Loss:   320.8549 Validation Accuracy: 0.804688
Epoch  6, Batch   6 - Loss:   528.4717 Validation Accuracy: 0.804688
Epoch  6, Batch   7 - Loss:   240.4545 Validation Accuracy: 0.804688
Epoch  6, Batch   8 - Loss:   392.0098 Validation Accuracy: 0.804688
Epoch  6, Batch   9 - Loss:   538.0865 Validation Accuracy: 0.804688
Epoch  6, Batch  10 - Loss:   500.1584 Validation Accuracy: 0.804688
Epoch  6, Batch  11 - Loss:   260.0012 Validation Accuracy: 0.804688
Epoch  6, Batch  12 - Loss:   358.

Epoch  6, Batch 117 - Loss:   566.1067 Validation Accuracy: 0.812500
Epoch  6, Batch 118 - Loss:   503.3497 Validation Accuracy: 0.808594
Epoch  6, Batch 119 - Loss:   384.1327 Validation Accuracy: 0.808594
Epoch  6, Batch 120 - Loss:   382.3274 Validation Accuracy: 0.808594
Epoch  6, Batch 121 - Loss:   261.3203 Validation Accuracy: 0.812500
Epoch  6, Batch 122 - Loss:   661.4438 Validation Accuracy: 0.808594
Epoch  6, Batch 123 - Loss:   339.5006 Validation Accuracy: 0.808594
Epoch  6, Batch 124 - Loss:   318.7048 Validation Accuracy: 0.808594
Epoch  6, Batch 125 - Loss:   396.0432 Validation Accuracy: 0.808594
Epoch  6, Batch 126 - Loss:   383.7574 Validation Accuracy: 0.808594
Epoch  6, Batch 127 - Loss:   260.6219 Validation Accuracy: 0.812500
Epoch  6, Batch 128 - Loss:   564.4866 Validation Accuracy: 0.804688
Epoch  6, Batch 129 - Loss:   542.9631 Validation Accuracy: 0.812500
Epoch  6, Batch 130 - Loss:   369.2386 Validation Accuracy: 0.808594
Epoch  6, Batch 131 - Loss:   284.

Epoch  6, Batch 236 - Loss:   243.5969 Validation Accuracy: 0.812500
Epoch  6, Batch 237 - Loss:   338.0996 Validation Accuracy: 0.812500
Epoch  6, Batch 238 - Loss:   333.2029 Validation Accuracy: 0.812500
Epoch  6, Batch 239 - Loss:   367.8258 Validation Accuracy: 0.812500
Epoch  6, Batch 240 - Loss:   418.6838 Validation Accuracy: 0.812500
Epoch  6, Batch 241 - Loss:   104.7226 Validation Accuracy: 0.812500
Epoch  6, Batch 242 - Loss:   521.6608 Validation Accuracy: 0.808594
Epoch  6, Batch 243 - Loss:   497.9378 Validation Accuracy: 0.812500
Epoch  6, Batch 244 - Loss:   486.8384 Validation Accuracy: 0.812500
Epoch  6, Batch 245 - Loss:   302.3748 Validation Accuracy: 0.812500
Epoch  6, Batch 246 - Loss:   549.5600 Validation Accuracy: 0.808594
Epoch  6, Batch 247 - Loss:   173.3426 Validation Accuracy: 0.808594
Epoch  6, Batch 248 - Loss:   497.4456 Validation Accuracy: 0.808594
Epoch  6, Batch 249 - Loss:   494.9492 Validation Accuracy: 0.808594
Epoch  6, Batch 250 - Loss:   401.

Epoch  6, Batch 355 - Loss:   257.6845 Validation Accuracy: 0.816406
Epoch  6, Batch 356 - Loss:   527.0450 Validation Accuracy: 0.808594
Epoch  6, Batch 357 - Loss:   530.1903 Validation Accuracy: 0.800781
Epoch  6, Batch 358 - Loss:   390.0240 Validation Accuracy: 0.808594
Epoch  6, Batch 359 - Loss:   347.2629 Validation Accuracy: 0.808594
Epoch  6, Batch 360 - Loss:   421.0457 Validation Accuracy: 0.812500
Epoch  6, Batch 361 - Loss:   309.7520 Validation Accuracy: 0.804688
Epoch  6, Batch 362 - Loss:   338.6744 Validation Accuracy: 0.804688
Epoch  6, Batch 363 - Loss:   275.1621 Validation Accuracy: 0.816406
Epoch  6, Batch 364 - Loss:   438.1200 Validation Accuracy: 0.808594
Epoch  6, Batch 365 - Loss:   308.1774 Validation Accuracy: 0.808594
Epoch  6, Batch 366 - Loss:   469.9041 Validation Accuracy: 0.808594
Epoch  6, Batch 367 - Loss:   453.5319 Validation Accuracy: 0.800781
Epoch  6, Batch 368 - Loss:   310.3943 Validation Accuracy: 0.800781
Epoch  6, Batch 369 - Loss:   383.

KeyboardInterrupt: 